In [1]:
from os import listdir
from PIL import Image
import numpy as np
import re
import pandas as pd


In [5]:
path = "data/matplotfaces"
ignore = ["morralla",".DS_Store","contempt","train","predict"]
files = [im for im in listdir(path) if im not in ignore]
HEIGHT,WIDTH = 64,64

In [9]:
len(files)

21152

In [7]:
y = [re.findall(".*_",x)[0].replace("_","")for x in files]

In [8]:
len(y)

21152

In [10]:
X = []
for f in files:
    img = Image.open(f"{path}/{f}").convert("L").resize((HEIGHT, WIDTH))
    X.append(np.array(img))

In [11]:
X =np.array(X)/255

In [12]:
X.shape

(21152, 64, 64)

In [13]:
len(y)

21152

In [14]:
cat = list(set(y))

In [15]:
cat

['fear', 'happiness', 'surprise', 'disgust', 'anger', 'neutral', 'sadness']

In [16]:
y=pd.DataFrame(y)

In [17]:
y

,0
0,neutral
1,happiness
2,neutral
3,happiness
4,happiness
...,...
21147,anger
21148,happiness
21149,happiness
21150,anger


In [29]:
y_dummies = pd.get_dummies(y, prefix="", prefix_sep='')

In [30]:
y_dummies

,anger,disgust,fear,happiness,neutral,sadness,surprise
0,0,0,0,0,1,0,0
1,0,0,0,1,0,0,0
2,0,0,0,0,1,0,0
3,0,0,0,1,0,0,0
4,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...
21147,1,0,0,0,0,0,0
21148,0,0,0,1,0,0,0
21149,0,0,0,1,0,0,0
21150,1,0,0,0,0,0,0


In [31]:
from sklearn.model_selection import train_test_split

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X,y_dummies)

In [33]:
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,BatchNormalization
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.optimizers import Adam
import keras

In [37]:
model = Sequential()
#model.add(Flatten())
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X.shape[1:])))
model.add(MaxPooling1D(pool_size=2))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Conv1D(filters=256, kernel_size=2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Conv1D(filters=512, kernel_size=2, activation='relu', input_shape=(X.shape[1:])))
model.add(MaxPooling1D(pool_size=2))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(64,activation = 'relu'))
model.add(Dense(128,activation = 'relu'))
model.add(Dense(256,activation = 'relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [46]:
from keras.models import Sequential, load_model, model_from_json
from keras import callbacks, optimizers
import tensorflow as tf
from datetime import date

fecha=str(date.today().year)+str(date.today().month)+str(date.today().day)
    
symbol = 'landmark_faces'
h5 = symbol + '_best_model' + fecha + '.h5'
checkpoint = callbacks.ModelCheckpoint(h5,
                                       monitor='loss',
                                       verbose=0,
                                       save_best_only=True,
                                       save_weights_only=True,
                                       mode='auto',
                                       period=1)
callback = [checkpoint]
json = symbol + '_best_model' + '.json'
model_json = model.to_json()
with open(json, "w") as json_file:
    json_file.write(model_json)

    
from IPython.display import clear_output
clear_output(wait=True)
modelo = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs = 5000,callbacks = callback,validation_split = 0.1)

Epoch 1/5000
447/447 [==============================] - 3s 6ms/step - loss: 1.4312 - accuracy: 0.4647 - val_loss: 1.4590 - val_accuracy: 0.4549
Epoch 2/5000
447/447 [==============================] - 3s 6ms/step - loss: 1.4218 - accuracy: 0.4656 - val_loss: 1.5946 - val_accuracy: 0.4184
Epoch 3/5000
447/447 [==============================] - 3s 6ms/step - loss: 1.4115 - accuracy: 0.4730 - val_loss: 1.4684 - val_accuracy: 0.4455
Epoch 4/5000
447/447 [==============================] - 3s 6ms/step - loss: 1.4047 - accuracy: 0.4766 - val_loss: 1.5926 - val_accuracy: 0.4241
Epoch 5/5000
447/447 [==============================] - 3s 6ms/step - loss: 1.3863 - accuracy: 0.4792 - val_loss: 1.4846 - val_accuracy: 0.4505
Epoch 6/5000
447/447 [==============================] - 3s 7ms/step - loss: 1.3911 - accuracy: 0.4759 - val_loss: 1.4723 - val_accuracy: 0.4512
Epoch 7/5000
447/447 [==============================] - 3s 7ms/step - loss: 1.3794 - accuracy: 0.4846 - val_loss: 1.4782 - val_accuracy:

447/447 [==============================] - 3s 6ms/step - loss: 1.0869 - accuracy: 0.5933 - val_loss: 1.5606 - val_accuracy: 0.4657
Epoch 58/5000
447/447 [==============================] - 3s 7ms/step - loss: 1.0788 - accuracy: 0.5963 - val_loss: 1.5841 - val_accuracy: 0.4638
Epoch 59/5000
447/447 [==============================] - 3s 7ms/step - loss: 1.0689 - accuracy: 0.6016 - val_loss: 1.5755 - val_accuracy: 0.4480
Epoch 60/5000
447/447 [==============================] - 3s 6ms/step - loss: 1.0729 - accuracy: 0.5953 - val_loss: 1.5410 - val_accuracy: 0.4745
Epoch 61/5000
447/447 [==============================] - 3s 7ms/step - loss: 1.0628 - accuracy: 0.6039 - val_loss: 1.5456 - val_accuracy: 0.4713
Epoch 62/5000
447/447 [==============================] - 3s 7ms/step - loss: 1.0513 - accuracy: 0.6087 - val_loss: 1.5663 - val_accuracy: 0.4549
Epoch 63/5000
447/447 [==============================] - 3s 7ms/step - loss: 1.0577 - accuracy: 0.6069 - val_loss: 1.6081 - val_accuracy: 0.4486

447/447 [==============================] - 3s 6ms/step - loss: 0.8913 - accuracy: 0.6656 - val_loss: 1.7156 - val_accuracy: 0.4600
Epoch 114/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.8922 - accuracy: 0.6709 - val_loss: 1.7191 - val_accuracy: 0.4518
Epoch 115/5000
447/447 [==============================] - 3s 7ms/step - loss: 0.8929 - accuracy: 0.6681 - val_loss: 1.7094 - val_accuracy: 0.4524
Epoch 116/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.8994 - accuracy: 0.6616 - val_loss: 1.7404 - val_accuracy: 0.4619
Epoch 117/5000
447/447 [==============================] - 3s 7ms/step - loss: 0.8800 - accuracy: 0.6711 - val_loss: 1.7320 - val_accuracy: 0.4423
Epoch 118/5000
447/447 [==============================] - 3s 7ms/step - loss: 0.8758 - accuracy: 0.6768 - val_loss: 1.7795 - val_accuracy: 0.4524
Epoch 119/5000
447/447 [==============================] - 3s 7ms/step - loss: 0.8817 - accuracy: 0.6694 - val_loss: 1.7348 - val_accuracy: 

447/447 [==============================] - 3s 6ms/step - loss: 0.7801 - accuracy: 0.7118 - val_loss: 1.8921 - val_accuracy: 0.4436
Epoch 170/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.7719 - accuracy: 0.7158 - val_loss: 1.8632 - val_accuracy: 0.4531
Epoch 171/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.7639 - accuracy: 0.7165 - val_loss: 1.9658 - val_accuracy: 0.4417
Epoch 172/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.7848 - accuracy: 0.7060 - val_loss: 1.8690 - val_accuracy: 0.4461
Epoch 173/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.7696 - accuracy: 0.7162 - val_loss: 1.8973 - val_accuracy: 0.4411
Epoch 174/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.7697 - accuracy: 0.7132 - val_loss: 1.8736 - val_accuracy: 0.4442
Epoch 175/5000
447/447 [==============================] - 3s 7ms/step - loss: 0.7655 - accuracy: 0.7170 - val_loss: 1.9088 - val_accuracy: 

447/447 [==============================] - 3s 6ms/step - loss: 0.6989 - accuracy: 0.7425 - val_loss: 1.9596 - val_accuracy: 0.4411
Epoch 226/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.6983 - accuracy: 0.7380 - val_loss: 1.9370 - val_accuracy: 0.4442
Epoch 227/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.7103 - accuracy: 0.7375 - val_loss: 1.9245 - val_accuracy: 0.4543
Epoch 228/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.6930 - accuracy: 0.7436 - val_loss: 1.9772 - val_accuracy: 0.4556
Epoch 229/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.7049 - accuracy: 0.7389 - val_loss: 1.9379 - val_accuracy: 0.4493
Epoch 230/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.6874 - accuracy: 0.7448 - val_loss: 2.0119 - val_accuracy: 0.4253
Epoch 231/5000
447/447 [==============================] - 3s 7ms/step - loss: 0.6921 - accuracy: 0.7436 - val_loss: 1.9550 - val_accuracy: 

...

447/447 [==============================] - 3s 6ms/step - loss: 0.5733 - accuracy: 0.7883 - val_loss: 2.1291 - val_accuracy: 0.4423
Epoch 450/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.5587 - accuracy: 0.7944 - val_loss: 2.1406 - val_accuracy: 0.4360
Epoch 451/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.5476 - accuracy: 0.8007 - val_loss: 2.1217 - val_accuracy: 0.4392
Epoch 452/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.5334 - accuracy: 0.8032 - val_loss: 2.2182 - val_accuracy: 0.4405
Epoch 453/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.5434 - accuracy: 0.7952 - val_loss: 2.2037 - val_accuracy: 0.4474
Epoch 454/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.5373 - accuracy: 0.8051 - val_loss: 2.1977 - val_accuracy: 0.4524
Epoch 455/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.5330 - accuracy: 0.8039 - val_loss: 2.1917 - val_accuracy: 

447/447 [==============================] - 3s 6ms/step - loss: 0.5242 - accuracy: 0.8082 - val_loss: 2.2051 - val_accuracy: 0.4373
Epoch 506/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.5321 - accuracy: 0.8055 - val_loss: 2.1878 - val_accuracy: 0.4568
Epoch 507/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.5266 - accuracy: 0.8100 - val_loss: 2.1998 - val_accuracy: 0.4386
Epoch 508/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.5131 - accuracy: 0.8108 - val_loss: 2.2488 - val_accuracy: 0.4291
Epoch 509/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.5213 - accuracy: 0.8048 - val_loss: 2.2476 - val_accuracy: 0.4386
Epoch 510/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.5258 - accuracy: 0.8094 - val_loss: 2.2688 - val_accuracy: 0.4512
Epoch 511/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.5221 - accuracy: 0.8073 - val_loss: 2.2777 - val_accuracy: 

447/447 [==============================] - 3s 6ms/step - loss: 0.4996 - accuracy: 0.8157 - val_loss: 2.2687 - val_accuracy: 0.4392
Epoch 562/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.5053 - accuracy: 0.8174 - val_loss: 2.3269 - val_accuracy: 0.4468
Epoch 563/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.5088 - accuracy: 0.8152 - val_loss: 2.2548 - val_accuracy: 0.4417
Epoch 564/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.5027 - accuracy: 0.8138 - val_loss: 2.2612 - val_accuracy: 0.4455
Epoch 565/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.5155 - accuracy: 0.8131 - val_loss: 2.2283 - val_accuracy: 0.4405
Epoch 566/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.5072 - accuracy: 0.8135 - val_loss: 2.2473 - val_accuracy: 0.4449
Epoch 567/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.5020 - accuracy: 0.8142 - val_loss: 2.2581 - val_accuracy: 

447/447 [==============================] - 3s 6ms/step - loss: 0.4989 - accuracy: 0.8168 - val_loss: 2.2628 - val_accuracy: 0.4398
Epoch 618/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.4819 - accuracy: 0.8266 - val_loss: 2.2829 - val_accuracy: 0.4461
Epoch 619/5000
447/447 [==============================] - 3s 7ms/step - loss: 0.4837 - accuracy: 0.8212 - val_loss: 2.2829 - val_accuracy: 0.4386
Epoch 620/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.5068 - accuracy: 0.8163 - val_loss: 2.2090 - val_accuracy: 0.4417
Epoch 621/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.4912 - accuracy: 0.8184 - val_loss: 2.2874 - val_accuracy: 0.4493
Epoch 622/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.4815 - accuracy: 0.8252 - val_loss: 2.3015 - val_accuracy: 0.4367
Epoch 623/5000
447/447 [==============================] - 3s 6ms/step - loss: 0.4950 - accuracy: 0.8180 - val_loss: 2.2688 - val_accuracy: 

KeyboardInterrupt: 